<a href="https://colab.research.google.com/github/Harjandar/absa-restaurant-sentiment/blob/main/notebooks/ABSA_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [141]:
# ==============================
# STEP 0: Import required libraries
# ==============================

import pandas as pd  # Data handling


In [142]:
# ==============================
# STEP 1: Load TRAIN dataset
# ==============================

train_url = "https://raw.githubusercontent.com/Harjandar/absa-restaurant-sentiment/main/data/raw/restaurants_train_single.csv"

train_df = pd.read_csv(train_url)

# Check dataset shape and top rows
print("Total rows in TRAIN dataset:", len(train_df))
train_df.head()


Total rows in TRAIN dataset: 2507


,sentence Id,sentence,aspect_category,aspect_term,from,to,polarity
0,1004293:0,Judging from previous posts this used to be a ...,RESTAURANT#GENERAL,place,51,56,negative
1,1004293:1,"We, there were four of us, arrived at noon - t...",SERVICE#GENERAL,staff,75,80,negative
2,1004293:2,"They never brought us complimentary noodles, i...",SERVICE#GENERAL,NaN,0,0,negative
3,1004293:3,The food was lousy - too sweet or too salty an...,FOOD#QUALITY,food,4,8,negative
4,1004293:3,The food was lousy - too sweet or too salty an...,FOOD#STYLE_OPTIONS,portions,52,60,negative


In [143]:
# ==============================
# STEP 2: Inspect all aspect categories (no filtering yet)
# ==============================

print("Original aspect categories and counts:")
print(train_df['aspect_category'].value_counts())


Original aspect categories and counts:
aspect_category
FOOD#QUALITY                849
SERVICE#GENERAL             449
RESTAURANT#GENERAL          422
AMBIENCE#GENERAL            255
FOOD#STYLE_OPTIONS          137
RESTAURANT#MISCELLANEOUS     98
FOOD#PRICES                  90
RESTAURANT#PRICES            80
DRINKS#QUALITY               47
DRINKS#STYLE_OPTIONS         32
LOCATION#GENERAL             28
DRINKS#PRICES                20
Name: count, dtype: int64


In [144]:
# ==============================
# STEP 3: Map old aspect categories to new simplified ones
# ==============================

aspect_mapping = {
    # FOOD related
    'FOOD#QUALITY': 'FOOD',
    'FOOD#STYLE_OPTIONS': 'FOOD',
    'FOOD#PRICES': 'FOOD',

    # SERVICE related
    'SERVICE#GENERAL': 'SERVICE',

    # RESTAURANT related → DELIVERY/OVERALL EXPERIENCE
    'RESTAURANT#GENERAL': 'DELIVERY_OVERALL',
    'RESTAURANT#PRICES': 'DELIVERY_OVERALL',
    'RESTAURANT#MISCELLANEOUS': 'DELIVERY_OVERALL'
}


In [145]:
# ==============================
# STEP 4: Keep only rows that exist in mapping
# ==============================

train_df = train_df[train_df['aspect_category'].isin(aspect_mapping.keys())]

# Verify
print("Rows after filtering:", len(train_df))
train_df['aspect_category'].value_counts()


Rows after filtering: 2125


,count
aspect_category,
FOOD#QUALITY,849
SERVICE#GENERAL,449
RESTAURANT#GENERAL,422
FOOD#STYLE_OPTIONS,137
RESTAURANT#MISCELLANEOUS,98
FOOD#PRICES,90
RESTAURANT#PRICES,80


In [146]:
# ==============================
# STEP 5: Add simplified aspect column
# ==============================

train_df['aspect_simplified'] = train_df['aspect_category'].map(aspect_mapping)

# Check
train_df[['aspect_category', 'aspect_simplified']].head()


,aspect_category,aspect_simplified
0,RESTAURANT#GENERAL,DELIVERY_OVERALL
1,SERVICE#GENERAL,SERVICE
2,SERVICE#GENERAL,SERVICE
3,FOOD#QUALITY,FOOD
4,FOOD#STYLE_OPTIONS,FOOD


In [147]:
# ==============================
# STEP 6: Check for missing values and drop if any
# ==============================

print("Missing values per column:")
print(train_df.isnull().sum())

# Drop missing rows (if any)
train_df = train_df.dropna()

print("Rows after dropping missing:", len(train_df))


Missing values per column:
sentence Id            0
sentence               0
aspect_category        0
aspect_term          591
from                   0
to                     0
polarity               0
aspect_simplified      0
dtype: int64
Rows after dropping missing: 1534


In [148]:
# Check final columns
train_df.head()


,sentence Id,sentence,aspect_category,aspect_term,from,to,polarity,aspect_simplified
0,1004293:0,Judging from previous posts this used to be a ...,RESTAURANT#GENERAL,place,51,56,negative,DELIVERY_OVERALL
1,1004293:1,"We, there were four of us, arrived at noon - t...",SERVICE#GENERAL,staff,75,80,negative,SERVICE
3,1004293:3,The food was lousy - too sweet or too salty an...,FOOD#QUALITY,food,4,8,negative,FOOD
4,1004293:3,The food was lousy - too sweet or too salty an...,FOOD#STYLE_OPTIONS,portions,52,60,negative,FOOD
6,1004293:5,Avoid this place!,RESTAURANT#GENERAL,place,11,16,negative,DELIVERY_OVERALL


In [149]:
# ==============================
# STEP 8: Combine sentence + aspect
# ==============================

# Create a new column for model input
train_df['model_input'] = train_df['sentence'] + " [ASP] " + train_df['aspect_simplified']

# Check first few rows
train_df[['sentence', 'aspect_simplified', 'model_input']].head()


,sentence,aspect_simplified,model_input
0,Judging from previous posts this used to be a ...,DELIVERY_OVERALL,Judging from previous posts this used to be a ...
1,"We, there were four of us, arrived at noon - t...",SERVICE,"We, there were four of us, arrived at noon - t..."
3,The food was lousy - too sweet or too salty an...,FOOD,The food was lousy - too sweet or too salty an...
4,The food was lousy - too sweet or too salty an...,FOOD,The food was lousy - too sweet or too salty an...
6,Avoid this place!,DELIVERY_OVERALL,Avoid this place! [ASP] DELIVERY_OVERALL


In [150]:
# ==============================
# STEP 9: Encode polarity labels
# ==============================

polarity_mapping = {
    'positive': 2,
    'neutral': 1,
    'negative': 0
}

train_df['polarity_encoded'] = train_df['polarity'].map(polarity_mapping)

# Check mapping
train_df[['polarity', 'polarity_encoded']].head()


,polarity,polarity_encoded
0,negative,0
1,negative,0
3,negative,0
4,negative,0
6,negative,0


In [151]:
# ==============================
# STEP 10: Basic text cleaning
# ==============================

def clean_text(text):
    text = text.lower()                    # lowercase
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # remove punctuation
    text = re.sub(r'\s+', ' ', text).strip()    # remove extra spaces
    return text

# Apply cleaning
train_df['model_input_clean'] = train_df['model_input'].apply(clean_text)

# Check
train_df[['model_input', 'model_input_clean']].head()


,model_input,model_input_clean
0,Judging from previous posts this used to be a ...,judging from previous posts this used to be a ...
1,"We, there were four of us, arrived at noon - t...",we there were four of us arrived at noon the p...
3,The food was lousy - too sweet or too salty an...,the food was lousy too sweet or too salty and ...
4,The food was lousy - too sweet or too salty an...,the food was lousy too sweet or too salty and ...
6,Avoid this place! [ASP] DELIVERY_OVERALL,avoid this place asp deliveryoverall


In [152]:
# ==============================
# Check polarity distribution BEFORE balancing
# ==============================

print("Polarity distribution before balancing:")
print(train_df['polarity'].value_counts())


Polarity distribution before balancing:
polarity
positive    1044
negative     438
neutral       52
Name: count, dtype: int64


In [153]:
# ==============================
# STEP 10b: Balance TRAIN dataset
# ==============================

from sklearn.utils import resample

# Separate each class
df_pos = train_df[train_df['polarity'] == 'positive']
df_neg = train_df[train_df['polarity'] == 'negative']
df_neu = train_df[train_df['polarity'] == 'neutral']

# Upsample neutral to match positive
df_neu_up = resample(df_neu,
                     replace=True,
                     n_samples=len(df_pos),   # match positive
                     random_state=42)

# Upsample negative to match positive
df_neg_up = resample(df_neg,
                     replace=True,
                     n_samples=len(df_pos),
                     random_state=42)

# Combine all classes
train_df_balanced = pd.concat([df_pos, df_neg_up, df_neu_up])

# Shuffle
train_df_balanced = train_df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

# Use balanced dataset for training
train_df = train_df_balanced

# Check distribution after balancing
print("Polarity distribution AFTER balancing:")
print(train_df['polarity'].value_counts())


Polarity distribution AFTER balancing:
polarity
negative    1044
neutral     1044
positive    1044
Name: count, dtype: int64


In [154]:
# ==============================
# STEP 11: Tokenization
# ==============================

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Hyperparameters
max_words = 5000      # maximum vocabulary size
max_len = 50          # maximum sequence length

# Create tokenizer
tokenizer = Tokenizer(num_words=max_words, oov_token='<OOV>')
tokenizer.fit_on_texts(train_df['model_input_clean'])

# Convert text to sequences
X_train_seq = tokenizer.texts_to_sequences(train_df['model_input_clean'])

# Pad sequences
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding='post', truncating='post')

print("Example tokenized sequence:", X_train_pad[0])
print("Shape of padded sequences:", X_train_pad.shape)


Example tokenized sequence: [ 19  22 210 132  81 501   3  15   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
Shape of padded sequences: (3132, 50)


In [155]:
# ==============================
# STEP 12: Prepare labels for LSTM
# ==============================

y_train = train_df['polarity_encoded'].values
print("Example labels:", y_train[:10])
print("Shape of labels:", y_train.shape)


Example labels: [0 1 2 2 0 1 1 0 0 2]
Shape of labels: (3132,)


In [156]:
# ==============================
# STEP 13: Load TEST dataset
# ==============================

test_url = "https://raw.githubusercontent.com/Harjandar/absa-restaurant-sentiment/main/data/raw/restaurants_test_single.csv"

test_df = pd.read_csv(test_url)

print("Total rows in TEST dataset:", len(test_df))
test_df.head()


Total rows in TEST dataset: 859


,sentence Id,sentence,aspect_category,aspect_term,from,to,polarity
0,en_BlueRibbonSushi_478218171:0,Yum!,FOOD#QUALITY,NaN,0,0,positive
1,en_BlueRibbonSushi_478218171:1,Serves really good sushi.,FOOD#QUALITY,sushi,19,24,positive
2,en_BlueRibbonSushi_478218171:2,Not the biggest portions but adequate.,FOOD#STYLE_OPTIONS,portions,16,24,neutral
3,en_BlueRibbonSushi_478218171:3,Green Tea creme brulee is a must!,FOOD#QUALITY,Green Tea creme brulee,0,22,positive
4,en_BlueRibbonSushi_478218171:4,Don't leave the restaurant without it.,FOOD#QUALITY,NaN,0,0,positive


In [157]:
# ==============================
# STEP 14: Filter test data for only relevant aspects
# ==============================

# Use the SAME aspect_mapping as train
test_df = test_df[test_df['aspect_category'].isin(aspect_mapping.keys())]

print("Rows after filtering TEST dataset:", len(test_df))
test_df['aspect_category'].value_counts()


Rows after filtering TEST dataset: 742


,count
aspect_category,
FOOD#QUALITY,313
SERVICE#GENERAL,155
RESTAURANT#GENERAL,142
FOOD#STYLE_OPTIONS,55
RESTAURANT#MISCELLANEOUS,33
FOOD#PRICES,23
RESTAURANT#PRICES,21


In [158]:
# ==============================
# Test data distribution
# ==============================

# Total rows in test set
print("Total rows in TEST dataset:", len(test_df))

# Count of each polarity
print("\nPolarity distribution in TEST dataset:")
print(test_df['polarity'].value_counts())


Total rows in TEST dataset: 742

Polarity distribution in TEST dataset:
polarity
positive    507
negative    196
neutral      39
Name: count, dtype: int64


In [159]:
# ==============================
# STEP 15: Map aspect_category → simplified aspect
# ==============================

test_df['aspect_simplified'] = test_df['aspect_category'].map(aspect_mapping)

test_df[['aspect_category', 'aspect_simplified']].head()


,aspect_category,aspect_simplified
0,FOOD#QUALITY,FOOD
1,FOOD#QUALITY,FOOD
2,FOOD#STYLE_OPTIONS,FOOD
3,FOOD#QUALITY,FOOD
4,FOOD#QUALITY,FOOD


In [160]:
# ==============================
# STEP 16: Combine sentence + aspect for model input
# ==============================

test_df['model_input'] = test_df['sentence'] + " [ASP] " + test_df['aspect_simplified']

# Basic text cleaning
test_df['model_input_clean'] = test_df['model_input'].apply(clean_text)

test_df[['sentence', 'aspect_simplified', 'model_input_clean']].head()


,sentence,aspect_simplified,model_input_clean
0,Yum!,FOOD,yum asp food
1,Serves really good sushi.,FOOD,serves really good sushi asp food
2,Not the biggest portions but adequate.,FOOD,not the biggest portions but adequate asp food
3,Green Tea creme brulee is a must!,FOOD,green tea creme brulee is a must asp food
4,Don't leave the restaurant without it.,FOOD,dont leave the restaurant without it asp food


In [161]:
# ==============================
# STEP 17: Encode polarity labels for TEST
# ==============================

test_df['polarity_encoded'] = test_df['polarity'].map(polarity_mapping)

# Prepare y_test
y_test = test_df['polarity_encoded'].values
y_test[:10]


array([2, 2, 1, 2, 2, 2, 2, 2, 2, 2])

In [162]:
# ==============================
# STEP 18: Tokenize test data
# ==============================

X_test_seq = tokenizer.texts_to_sequences(test_df['model_input_clean'])
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len, padding='post', truncating='post')

print("Shape of X_test_pad:", X_test_pad.shape)


Shape of X_test_pad: (742, 50)


In [163]:
# ==============================
# STEP 19: Import required Keras modules
# ==============================

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping


In [165]:
# ==============================
# STEP 20: Hyperparameters
# ==============================

vocab_size = 5000       # Same as tokenizer num_words
embedding_dim = 100     # Size of word embeddings
max_len = 50            # Sequence length (same as padding)
lstm_units = 128        # LSTM neurons
dropout_rate = 0.3      # Dropout for regularization
num_classes = 3         # polarity: positive / neutral / negative
learning_rate = 0.001


In [166]:
# ==============================
# STEP 21: Build LSTM model
# ==============================

model = Sequential()

# Embedding layer converts word indices → dense vectors
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_len))

# Bidirectional LSTM layer
model.add(Bidirectional(LSTM(lstm_units, return_sequences=False)))

# Dropout for regularization
model.add(Dropout(dropout_rate))

# Dense output layer with softmax for 3 classes
model.add(Dense(num_classes, activation='softmax'))

# Compile model
model.compile(
    loss='sparse_categorical_crossentropy',  # suitable for integer labels
    optimizer=Adam(learning_rate=learning_rate),
    metrics=['accuracy']
)

# Model summary
model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_4 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [167]:
# ==============================
# STEP 22: Train model
# ==============================

# Early stopping to prevent overfitting
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(
    X_train_pad, y_train,
    epochs=10,                 # You can increase later
    batch_size=32,
    validation_split=0.1,      # 10% train → validation
    callbacks=[early_stop],
    verbose=1
)


Epoch 1/10
89/89 ━━━━━━━━━━━━━━━━━━━━ 19s 166ms/step - accuracy: 0.4726 - loss: 1.0073 - val_accuracy: 0.7898 - val_loss: 0.5364
Epoch 2/10
89/89 ━━━━━━━━━━━━━━━━━━━━ 14s 159ms/step - accuracy: 0.8119 - loss: 0.4465 - val_accuracy: 0.8726 - val_loss: 0.3289
Epoch 3/10
89/89 ━━━━━━━━━━━━━━━━━━━━ 14s 157ms/step - accuracy: 0.9344 - loss: 0.2150 - val_accuracy: 0.9204 - val_loss: 0.2467
Epoch 4/10
89/89 ━━━━━━━━━━━━━━━━━━━━ 14s 158ms/step - accuracy: 0.9530 - loss: 0.1581 - val_accuracy: 0.9140 - val_loss: 0.2425
Epoch 5/10
89/89 ━━━━━━━━━━━━━━━━━━━━ 22s 173ms/step - accuracy: 0.9569 - loss: 0.1348 - val_accuracy: 0.8917 - val_loss: 0.2803
Epoch 6/10
89/89 ━━━━━━━━━━━━━━━━━━━━ 14s 159ms/step - accuracy: 0.9656 - loss: 0.1074 - val_accuracy: 0.9236 - val_loss: 0.2067
Epoch 7/10
89/89 ━━━━━━━━━━━━━━━━━━━━ 20s 158ms/step - accuracy: 0.9676 - loss: 0.1032 - val_accuracy: 0.9172 - val_loss: 0.2385
Epoch 8/10
89/89 ━━━━━━━━━━━━━━━━━━━━ 14s 160ms/step - accuracy: 0.9676 - loss: 0.1143 - val_accu

In [168]:
# ==============================
# STEP 23: Evaluate model on TEST data
# ==============================

test_loss, test_acc = model.evaluate(X_test_pad, y_test, verbose=1)
print(f"✅ Test Accuracy: {test_acc:.4f}")


24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - accuracy: 0.7466 - loss: 0.6963
✅ Test Accuracy: 0.7412


In [169]:
# ==============================
# STEP 24: Predict polarity for new examples
# ==============================

pred_probs = model.predict(X_test_pad)
pred_classes = pred_probs.argmax(axis=1)

# Compare first 10 predictions vs true labels
for i in range(10):
    print(f"Sentence: {test_df['sentence'].iloc[i]}")
    print(f"Aspect: {test_df['aspect_simplified'].iloc[i]}")
    print(f"True: {y_test[i]}, Predicted: {pred_classes[i]}")
    print("-----")


24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 51ms/step
Sentence: Yum!
Aspect: FOOD
True: 2, Predicted: 2
-----
Sentence: Serves really good sushi.
Aspect: FOOD
True: 2, Predicted: 2
-----
Sentence: Not the biggest portions but adequate.
Aspect: FOOD
True: 1, Predicted: 0
-----
Sentence: Green Tea creme brulee is a must!
Aspect: FOOD
True: 2, Predicted: 2
-----
Sentence: Don't leave the restaurant without it.
Aspect: FOOD
True: 2, Predicted: 2
-----
Sentence: No Comparison
Aspect: DELIVERY_OVERALL
True: 2, Predicted: 0
-----
Sentence: – I can't say enough about this place.
Aspect: DELIVERY_OVERALL
True: 2, Predicted: 2
-----
Sentence: It has great sushi and even better service.
Aspect: FOOD
True: 2, Predicted: 2
-----
Sentence: It has great sushi and even better service.
Aspect: SERVICE
True: 2, Predicted: 2
-----
Sentence: The entire staff was extremely accomodating and tended to my every need.
Aspect: SERVICE
True: 2, Predicted: 2
-----


In [172]:
# ==============================
# STEP 24: Simple metrics printing
# ==============================

# ==============================
# Overall LSTM Test Performance
# ==============================

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Use pred_classes (not y_pred)
accuracy = accuracy_score(y_test, pred_classes)
precision = precision_score(y_test, pred_classes, average='macro', zero_division=0)
recall = recall_score(y_test, pred_classes, average='macro', zero_division=0)
f1 = f1_score(y_test, pred_classes, average='macro', zero_division=0)

print("✅ LSTM Test Performance")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-score: {f1}")





✅ LSTM Test Performance
Accuracy: 0.7412398921832885
Precision: 0.6034674224719474
Recall: 0.535170873082961
F1-score: 0.5470234507939032


In [177]:
# ==============================
# Predict sentiment for each aspect in a review
# ==============================

def predict_aspect_sentiment_detailed(review, aspects_list):
    """
    Predict sentiment for each aspect in a review using trained LSTM model.

    Args:
        review (str): Full review text
        aspects_list (list): List of aspects to check (e.g., ["FOOD", "SERVICE", "DELIVERY_OVERALL"])

    Returns:
        dict: Each aspect with predicted sentiment and review snippet
    """
    result = {}

    # Map encoded labels back to text
    polarity_map_rev = {0:'Negative', 1:'Neutral', 2:'Positive'}

    for aspect in aspects_list:
        # 1️⃣ Combine sentence + aspect like in training
        input_text = review + " [ASP] " + aspect

        # 2️⃣ Clean text
        input_text_clean = clean_text(input_text)

        # 3️⃣ Tokenize
        seq = tokenizer.texts_to_sequences([input_text_clean])

        # 4️⃣ Pad sequence
        pad_seq = pad_sequences(seq, maxlen=max_len, padding='post')

        # 5️⃣ Predict class
        pred_class = model.predict(pad_seq, verbose=0).argmax(axis=1)[0]

        # 6️⃣ Map to polarity
        predicted_sentiment = polarity_map_rev[pred_class]

        # 7️⃣ Store result
        result[aspect] = {
            "sentiment": predicted_sentiment,
            "snippet": review  # full review as snippet
        }

    return result


In [178]:
# Example review
review = "rice was fantastic and tasty but service was bad"
# List of aspects you want to predict
aspects = ["FOOD", "SERVICE", "DELIVERY_OVERALL"]

# Call function with both arguments
result = predict_aspect_sentiment_detailed(review, aspects)

print(result)


{'FOOD': {'sentiment': 'Positive', 'snippet': 'rice was fantastic and tasty but service was bad'}, 'SERVICE': {'sentiment': 'Negative', 'snippet': 'rice was fantastic and tasty but service was bad'}, 'DELIVERY_OVERALL': {'sentiment': 'Negative', 'snippet': 'rice was fantastic and tasty but service was bad'}}
